<a href="https://colab.research.google.com/github/genfuru011/AD_Tech_SLM/blob/main/AD_Tech_SLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AD_Tech_SLM
広告特化型 SLM（Small Language Model）開発・運用プロジェクト

## プロジェクト概要

本プロジェクトは、“広告コピーの品質向上”と“CTR（クリック率）改善”を目指す、広告業界向けの特化型言語モデルを開発・運用するものです。  
データ収集からモデル開発・評価・デプロイ・運用まで、一貫したワークフローを設計し、効率的かつ安定的な価値提供を目指します。

---

## ロードマップ

### 短期（2ヶ月）：PoC → 本格トレーニング
### 中期（4ヶ月）：デプロイ＋運用パイプライン構築
### 長期（6ヶ月以降）：継続的改善・新機能展開

---

## チューニング手法

### 人間のフィードバックを活用（RLHF, DPO手法）
- 人間のフィードバックをもとに、より最適なコピーを生成するようモデルを学習させる
- 感情制御、要約生成、対話タスクなど、様々なタスクでRLHFと同等かそれ以上の人間選好一致性を示すことが報告されています。特に、トーンやスタイル、特定のコンテンツ指向など、主観的な要素が重要となる場合に効果を発揮します。
- 特に、トーンやスタイル、特定のコンテンツ指向など、主観的な要素が重要となる場合に効果を発揮します。
- 人間が好む出力と好まない出力のペアデータが不可欠です。このデータの質や量、多様性がモデルの性能に大きく影響します

### 必要データアセットJSONL
```json
{"prompt": "【テーマ】雨の日でもワクワクするニュースアプリを紹介してください",
 "chosen": "雨が降っても最新トレンドをスマホでサクッとチェック！天気と話題を同時にキャッチして、移動中も退屈知らず♪",
 "rejected": "ニュースが見られるよ！便利！"}

{"prompt": "【テーマ】忙しい会社員が移動中に英語を学べるアプリを紹介してください",
 "chosen": "通勤電車で 3 分！AI レッスンがあなたの発音を即フィードバック💡スキマ時間で着実にスキルアップ！",
 "rejected": "英語学習ができます。おすすめです。"}

{"prompt": "【テーマ】節電をサポートする家計簿アプリを紹介してください",
 "chosen": "家電ごとの電気代を自動計算！グラフでムダを一目で発見して、月末の請求額にもうドキドキしません✨",
 "rejected": "節約に役立つアプリです。ぜひ使ってください。"}
```
- prompt：コピーを書かせたいお題や前提条件をまとめます。
- chosen：CTR が高かった “良い広告テキスト” を入れます。
- rejected：CTR が低かった、または不採用になった文を入れます。

# settings command


## **ライブラリのインストール**

# =========================================
# 1. ライブラリのインストール
# -----------------------------------------

In [ ]:
import torch, platform, os, subprocess, textwrap, json, sys
print("Python   :", platform.python_version())
print("CUDA     :", torch.version.cuda)
!nvidia-smi -L    # A100 40GB であることを確認


In [ ]:
# ===========================================================
# 0. 依存パッケージ（最新版）        ★初回だけ実行で OK
# -----------------------------------------------------------
!pip install -q --upgrade \
  trl accelerate transformers peft bitsandbytes datasets sentencepiece

In [ ]:
# ===========================================================
# 1. Drive をマウントしてチェックポイント置き場を決定
# -----------------------------------------------------------
from google.colab import drive, runtime, output
drive.mount("/content/drive")
CKPT_DIR = "/content/drive/MyDrive/hf_ckpt/hameln_dpo_lora"

In [ ]:
# ===========================================================
# 2. Hugging Face 認証   ★Hameln 利用規約に同意済みトークン
# -----------------------------------------------------------
from huggingface_hub import login
login()

In [ ]:
# ===========================================================
# 3. モデル & トークナイザを 4-bit でロード
# -----------------------------------------------------------
import torch, os, glob
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

BASE_ID = "Elizezen/Hameln-japanese-mistral-7B"

bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="bfloat16",
    bnb_4bit_use_double_quant=True,
)
tok = AutoTokenizer.from_pretrained(BASE_ID, use_fast=False)
tok.pad_token = tok.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_ID,
    quantization_config=bnb_cfg,
    device_map="auto",
    trust_remote_code=True,
)


In [ ]:
# ===========================================================
# 4. LoRA 設定
# -----------------------------------------------------------
from peft import LoraConfig
lora_cfg = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj",
                    "gate_proj","up_proj","down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
# ===========================================================
# 5. データセット読み込み（HF Hub から直 DL）
# -----------------------------------------------------------
from datasets import load_dataset, load_from_disk
import os, glob

# ❶ Drive にキャッシュを作成して再利用
DS_CACHE = "/content/drive/MyDrive/hf_cache/anthropic_jp"

if os.path.exists(DS_CACHE):
    print("▶ Drive キャッシュから即時ロード …")
    train_ds = load_from_disk(DS_CACHE)
else:
    print("▶ Hugging Face Hub からダウンロード中 …")
    # デフォルト split は train（161k 行）
    train_ds = load_dataset("shi3z/anthropic_hh_rlhf_japanese", split="train")
    # 選好列はすでに chosen / rejected で揃っているのでリネーム不要 :contentReference[oaicite:0]{index=0}
    train_ds.save_to_disk(DS_CACHE)
    print("✅ Drive に保存しました →", DS_CACHE)

print("データ行数:", len(train_ds), "/ 列:", train_ds.column_names)


In [ ]:
# ===========================================================
# 6. DPOConfig ─ こまめ保存 + メモリ削減
# -----------------------------------------------------------
from trl import DPOConfig

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"  # 断片化回避

dpo_cfg = DPOConfig(
    output_dir              = CKPT_DIR,
    per_device_train_batch_size = 1,      # メモリ節約
    gradient_accumulation_steps = 8,      # 実バッチ 8
    gradient_checkpointing      = True,   # 勾配 CP で 30% 節約
    num_train_epochs            = 1,
    learning_rate               = 2e-4,
    beta                        = 0.1,
    logging_steps               = 50,
    save_steps                  = 200,    # 200 ステップごとに Drive へ保存
    eval_steps                  = 200,
    bf16                        = True,   # A100 用
)

In [ ]:
# ===========================================================
# 7. 最新チェックポイントを自動検出
# -----------------------------------------------------------
ckpts = sorted(glob.glob(f"{CKPT_DIR}/checkpoint-*"), key=os.path.getmtime)
resume_ckpt = ckpts[-1] if ckpts else None
print("再開チェックポイント:", resume_ckpt)


In [ ]:
# ===========================================================
# 8. DPOTrainer 生成 & 学習開始（参照モデルは CPU へ退避）
# -----------------------------------------------------------
from trl import DPOTrainer
trainer = DPOTrainer(
    model                 = base_model,
    ref_model             = BASE_ID,               # 文字列指定で自動ロード
    ref_model_init_kwargs = {"device_map":"cpu",
                             "load_in_4bit":True,
                             "low_cpu_mem_usage":True},
    args                  = dpo_cfg,
    processing_class      = tok,
    train_dataset         = train_ds,
    peft_config           = lora_cfg,              # そのまま渡す
    resume_from_checkpoint= resume_ckpt,
)
trainer.train()

In [ ]:
# ===========================================================
# 9. 学習完了後の LoRA アダプタ保存
# -----------------------------------------------------------
trainer.model.save_pretrained(CKPT_DIR + "/final-lora")
tok.save_pretrained(CKPT_DIR + "/final-lora")
print("✅ すべて完了！（LoRA は Drive に保存済み）")